## Working with Shapes

Here we explore how to visualize information using the cell morphology

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h, gui
import libcell

cell = libcell.L23()

In [ ]:
ps = h.Shape()

# ps = h.Shape(0)
# ps.view(-191.578, -172.628, 369.697, 431.069, 106, 253, 483.84, 563.2)

In [ ]:
ps.show(0)
# ps.show(1)

In [ ]:
ic = h.IClamp(cell.soma(0.5))

In [ ]:
ps.point_mark(ic, 3)

In [ ]:
ic.loc(cell.dends[30](1))
ic.get_segment()

In [ ]:
ps.point_mark_remove()

In [ ]:
ps = h.PlotShape(0)
ps.view(-191.578, -172.628, 369.697, 431.069, 106, 253, 483.84, 563.2)

In [ ]:
ps.variable("v")
ps.exec_menu("Shape Plot")

In [ ]:
ps.scale(-70, 0)

In [ ]:
ic.loc(cell.soma(0.5))

ic.delay = 0
ic.amp = 0.1
ic.dur = 100

h.v_init = -75
h.tstop = 100

h.init()
h.run()
ps.flush()

In [ ]:
vs = []
for sec in h.allsec():
    for seg in sec.allseg():
        vs.append(seg.v)
        
vsmin = np.min(vs)
vsmax = np.max(vs)
print(f'min voltage: {vsmin}, max voltage: {vsmax}')

In [ ]:
ps.scale(vsmin, vsmax)

In [ ]:
ps.printfile("L23_voltage_1.eps")

You can plot other variables, too, with a little trick.

In [ ]:
vs = []
for sec in h.allsec():
    for seg in sec.allseg():
        seg.v = h.distance(cell.soma(0.5), seg) # set the distance for soma
        vs.append(seg.v)
        
vsmin = np.min(vs)
vsmax = np.max(vs)
print(f'min voltage: {vsmin}, max voltage: {vsmax}')

ps.scale(vsmin, vsmax)

# Building a network

Network construction in NEURON follows the following steps:

1. Prepare neurons (morphology, mechanisms, etc.) and external stimuli if necessary,
2. Embed synapse objects in each neuron,
3. Connect the neurons to the synapses via [NetCon](https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/network/netcon.html),
4. Set up recordings,
5. Run the simulation.

![](./images/net_construction.png)


Here we demonstrate two simple networks with one pyramidal neuron and one or two external stimuli.

## 1 Pyramidal cell + 1 synapse

Again, we start from importing the NEURON and cell module.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h, gui
import libcell

cell = libcell.L23()

Here we open a session that embeds a synapse object called [Exp2Syn](https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/mech.html?highlight=exp2syn#Exp2Syn).

In [ ]:
syn = [h.Exp2Syn(cell.soma(0.5))]

In [ ]:
print(f'Erev = {syn[0].e}\ntau1 = {syn[0].tau1} ms\ntau2 = {syn[0].tau2} ms')

Again we define our run function:

In [ ]:
def run(tstop=250):
    """runs a simulation and returns the membrane potential recording at the soma."""
    dt = 0.1 # Again we use 10kHz sampling rate, e.g. 0.1 ms interval
    
    trec = h.Vector() # Record time
    vrec = h.Vector() # Record voltage
    
    trec.record(h._ref_t, dt)
    vrec.record(cell.soma(0.5)._ref_v, dt)
    
    h.v_init = -75
    
    h.tstop = tstop  # Set how long the simulation will run.
    h.init()
    h.run()
    
    return trec.c(), vrec.c() # Should return a copy of the vector

However, nothing happens during our simulation since there is no input connected to the synapse.

In [ ]:
t, vrec = run()

fig, ax = plt.subplots()
ax.plot(t, vrec)

Therefore, we prepare an artificial spike generator called [NetStim](https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/mech.html?highlight=netstim#NetStim) and connect this to this synapse by a `NetCon` object.

In [ ]:
# Check out the documentation for what the parameters are
stim = h.NetStim()
stim.start = 50
stim.number = 1
stim.interval = 5
stim.noise = 0

# Connect stim to our synapse
nc = h.NetCon(stim, syn[0])

Nothing happens yet since the synaptic conductance is 0. We set this to a non-zero value:

In [ ]:
nc.weight[0] = 0.01 # 0.01 nS = 10 pS of the synaptic conductance

In [ ]:
t, vrec = run()

fig, ax = plt.subplots()
ax.plot(t, vrec)
ax.set(xlabel="time (ms)", ylabel="voltage (mV)")

Now we see EPSPs triggered by spikes from `stim`.

In [ ]:
stim.start = 50
stim.number = 5
stim.interval = 40
stim.noise = 0

## Multiple synapses

Of course, we can add more synapses!

In [ ]:
syn.append(h.Exp2Syn(cell.soma(0.5)))

Here we put the spike generators and NetCons in lists:

In [ ]:
stim = []
nc = []
for i in range(2):
    stim.append(h.NetStim())
    stim[i].start = 50
    stim[i].number = 1
    stim[i].noise = 0
    
    nc.append(h.NetCon(stim[i], syn[i]))
    nc[i].weight[0] = 0.01

In [ ]:
stim[1].start = 150

t, vrec = run()

fig, ax = plt.subplots()
t = np.arange(vrec.size())*0.1
ax.plot(t, vrec)
ax.set(xlabel="time (ms)", ylabel="voltage (mV)")

Here we move the synapses:

In [ ]:
syn[0].loc(cell.dends[10](0.2))
syn[1].loc(cell.dends[10](0.8))

Let's check the locations!

In [ ]:
ps = h.Shape(0)
ps.view(-191.578, -172.628, 369.697, 431.069, 106, 253, 483.84, 563.2)

In [ ]:
ps.point_mark(syn[0], 2) # red
ps.point_mark(syn[1], 3) # blue

In [ ]:
stim[1].start = stim[0].start+15

t, vrec = run()

fig, ax = plt.subplots()
ax.plot(t, vrec)
ax.set(xlabel="time (ms)", ylabel="voltage (mV)")

Let's try different locations

In [ ]:
syn[0].loc(cell.dends[10](0.25))
syn[1].loc(cell.dends[11](0.8))

In [ ]:
stim[1].start = stim[0].start+15

t, vrec = run()

fig, ax = plt.subplots()
ax.plot(t, vrec)
ax.set(xlabel="time (ms)", ylabel="voltage (mV)")